## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar10_resnet56_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## (Resnet56) model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarResnetConfig(num_layers = 56, cifar = 10)
model = model_gen.GetCifarResnet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 1.577850, Test Acc: 51.24 %
2 epoch end, loss: 1.058704, Test Acc: 66.81 %
3 epoch end, loss: 0.838358, Test Acc: 68.07 %
4 epoch end, loss: 0.707965, Test Acc: 74.57 %
5 epoch end, loss: 0.622400, Test Acc: 76.34 %
6 epoch end, loss: 0.572558, Test Acc: 79.07 %
7 epoch end, loss: 0.524222, Test Acc: 76.08 %
8 epoch end, loss: 0.491849, Test Acc: 82.03 %
9 epoch end, loss: 0.462730, Test Acc: 80.30 %
10 epoch end, loss: 0.436930, Test Acc: 83.42 %
11 epoch end, loss: 0.416748, Test Acc: 83.66 %
12 epoch end, loss: 0.399653, Test Acc: 82.94 %
13 epoch end, loss: 0.380174, Test Acc: 84.41 %
14 epoch end, loss: 0.366987, Test Acc: 85.04 %
15 epoch end, loss: 0.352355, Test Acc: 84.89 %
16 epoch end, loss: 0.342741, Test Acc: 84.02 %
17 epoch end, loss: 0.331320, Test Acc: 85.51 %
18 epoch end, loss: 0.320803, Test Acc: 86.36 %
19 epoch end, loss: 0.308369, Test Acc: 82.03 %
20 epoch end, loss: 0.300488, Test Acc: 84.99 %
21 epoch end, loss: 0.294801, Test Acc: 84.53 %
2

171 epoch end, loss: 0.001954, Test Acc: 92.89 %
172 epoch end, loss: 0.002044, Test Acc: 92.87 %
173 epoch end, loss: 0.001984, Test Acc: 92.90 %
174 epoch end, loss: 0.002026, Test Acc: 92.90 %
175 epoch end, loss: 0.001781, Test Acc: 92.97 %
176 epoch end, loss: 0.001791, Test Acc: 92.97 %
177 epoch end, loss: 0.001791, Test Acc: 93.01 %
178 epoch end, loss: 0.001716, Test Acc: 93.01 %
179 epoch end, loss: 0.001609, Test Acc: 92.92 %
180 epoch end, loss: 0.001703, Test Acc: 92.97 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 92.97 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)